**Decoding the Secret message**

The objective is to write a function that takes in the URL for a Google Doc as an argument, retrieves and parses the data in the document, and prints the grid of characters. When printed in a fixed-width font, the characters in the grid will form a graphic showing a sequence of uppercase letters, which is the secret message.

In [10]:
import requests
import pandas as pd
from io import StringIO

In [15]:
# Get the HTML content from Google Doc
response = requests.get('https://docs.google.com/document/d/e/2PACX-1vQGUck9HIFCyezsrBSnmENk5ieJuYwpt7YHYEzeNJkIb9OSDdx-ov2nRNReKQyey-cwJOoEKUhLmN9z/pub')
html_content = response.text

In [17]:
# Wrap the HTML content in a StringIO object
html_io = StringIO(html_content)

In [22]:
if not tables:
  print("No tables found in the document.")

tables = pd.read_html(html_io)
data = tables[0]
data

,0,1,2
0,x-coordinate,Character,y-coordinate
1,93,░,5
2,2,░,5
3,8,░,0
4,39,█,1
...,...,...,...
346,27,█,1
347,61,█,3
348,28,░,4
349,14,█,1


In [24]:
# Check if the first row contains headers
if data.iloc[0].tolist() == ['x-coordinate', 'Character', 'y-coordinate']:
  # If headers are present, set them as column names and remove the first row
  data.columns = data.iloc[0]
  data = data.iloc[1:].reset_index(drop=True)

In [25]:
# Ensure we have the expected columns
if set(data.columns) != set(['x-coordinate', 'Character', 'y-coordinate']):
  print(f"Error: Unexpected column names. Found: {data.columns.tolist()}")

In [26]:
# Convert coordinates to integers
data['x-coordinate'] = pd.to_numeric(data['x-coordinate'], errors='coerce').fillna(0).astype(int)
data['y-coordinate'] = pd.to_numeric(data['y-coordinate'], errors='coerce').fillna(0).astype(int)

In [27]:
# Extract coordinates and characters
max_x = data['x-coordinate'].max()
max_y = data['y-coordinate'].max()
grid = [[' ' for _ in range(max_x + 1)] for _ in range(max_y + 1)]

In [28]:
# Populate the grid with characters
for i, row in data.iterrows():
  x, y, char = row['x-coordinate'], row['y-coordinate'], row['Character']
  grid[y][x] = char

In [29]:
# Print the grid (inverted y-axis)
for row in reversed(grid):
    print(''.join(row))

████████░     ████████░   ██████████░    ███████░  ██░           ███░ ███░    ███░ ██░     ██░
██░     ██░ ███░     ███░ ██░          ███░    ██░ ███░   ███░   ██░    ██░  ██░   ██░     ██░
██░     ██░ ██░       ██░ ██░         ███░          ██░  █████░ ███░     ██░██░    ██░     ██░
████████░   ██░       ██░ ████████░   ██░           ███░ ██░██░ ██░       ███░     ██████████░
██░     ██░ ██░       ██░ ██░         ███░           ██░██░ ██░██░       ██░██░    ██░     ██░
██░     ██░ ███░     ███░ ██░          ███░    ██░   ████░   ████░      ██░  ██░   ██░     ██░
████████░     ████████░   ██████████░    ███████░     ██░     ██░     ███░    ███░ ██░     ██░
